In [1]:
import torch as t
from torch import nn
import numpy as np
from tqdm.notebook import tqdm
import torchopt
import functorch
from functools import partial
from torch.utils.data import DataLoader, Dataset, RandomSampler


import os, sys
HOME = os.environ['HOME']  # change if necessary
sys.path.append(f'{HOME}/Finite-groups/src')
from model import MLP3, MLP4, InstancedModule
from utils import *
from group_data import *
from model_utils import *
from train import Parameters
%load_ext autoreload
%autoreload 2

Syntax warning: Unbound global variable in /usr/share/gap/pkg/browse/PackageIn\
fo.g:73
  if not IsKernelExtensionAvailable("Browse", "ncurses") then
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
Syntax warning: Unbound global variable in /usr/share/gap/pkg/cddinterface/Pac\
kageInfo.g:85
    if not IsKernelExtensionAvailable("CddInterface") then
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
Syntax warning: Unbound global variable in /usr/share/gap/pkg/crypting/Package\
Info.g:82
   if not IsKernelExtensionAvailable("crypting") then
          ^^^^^^^^^^^^^^^^^^^^^^^^^^
Syntax warning: Unbound global variable in /usr/share/gap/pkg/curlinterface/Pa\
ckageInfo.g:114
  if not IsKernelExtensionAvailable("curlinterface", "curl") then
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
Syntax warning: Unbound global variable in /usr/share/gap/pkg/cvec/PackageInfo\
.g:114
  if not IsKernelExtensionAvailable("cvec") then
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
Syntax warning: Unbound global variable in /usr/share/gap/pkg/datastruct

### Quadratic toy example

In [57]:
class QuadraticModel(nn.Module):
    def __init__(self, A):
        super().__init__()
        self.A = A
        self.x = nn.Parameter(t.randn(A.shape[0]))

    def forward(self):
        return (self.x.T @ self.A @ self.x).squeeze()

In [71]:
ADAM_CFG = {
    'lr': 0.01,
}
B = t.randn(5, 5)
model = QuadraticModel(B.T @ B + 0.01 * t.eye(5))
opt = torchopt.FuncOptimizer(torchopt.adam(**ADAM_CFG))
model_f, init_params = functorch.make_functional(model)

def train(init_params, iters):
    params = init_params
    for _ in tqdm(range(iters)):
        loss = model_f(params)
        params = opt.step(loss, params)
    return params

/tmp/ipykernel_16905/538059995.py:7: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.make_functional` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.func.functional_call` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  model_f, init_params = functorch.make_functional(model)


In [73]:
t.func.jacrev(partial(train, iters=10))(init_params)

  0%|          | 0/10 [00:00<?, ?it/s]

((tensor([[ 0.8728, -0.0568, -0.0088,  0.0194,  0.0596],
          [-0.0096,  0.9777, -0.0029,  0.0046,  0.0027],
          [-0.0029, -0.0059,  0.9853,  0.0128,  0.0014],
          [ 0.0054,  0.0075,  0.0104,  0.9740, -0.0010],
          [ 0.2598,  0.0741,  0.0190, -0.0174,  0.8710]],
         grad_fn=<ViewBackward0>),),)

### Groups

In [2]:
PARAMS = Parameters(
    instances=1,
    embed_dim=32,
    hidden_size=32,
    group_string='S(4)',
    model='MLP2',
    unembed_bias=True,
    weight_decay=2e-5,
    train_frac=0.4,
)

t.manual_seed(PARAMS.seed)
np.random.seed(PARAMS.seed)
random.seed(PARAMS.seed)
group_dataset = GroupData(params=PARAMS)
model = MODEL_DICT[PARAMS.model](params=PARAMS).to(device)

batch_size = len(group_dataset)
# sampler = RandomSampler(group_dataset, replacement=PARAMS.replacement)
train_loader = DataLoader(
    dataset=group_dataset,
    batch_size=batch_size,
    # shuffle=True,
    drop_last=True,
    # sampler=sampler,
)

# TODO: bias params should not get weight decay (to match with train.py)
# But probably doesn't matter much
opt = torchopt.FuncOptimizer(
    torchopt.adam(
        weight_decay=PARAMS.weight_decay,
        lr=PARAMS.lr,
        betas=[PARAMS.beta1, PARAMS.beta2],
    )
)
model_f, init_params = functorch.make_functional(model)
param_shapes = [p.shape for p in init_params]

def flatten(params):
    return t.cat([p.flatten() for p in params])

def unflatten(flat_params, shapes):
    params = []
    i = 0
    for shape in shapes:
        size = np.prod(shape)
        params.append(flat_params[i:i+size].reshape(shape))
        i += size
    return tuple(params)

def train(flat_init_params, epochs):
    params = unflatten(flat_init_params, param_shapes)
    for epoch in tqdm(range(epochs)):
        for x, z in train_loader:
            x = x.to(device)
            z = z.to(device)
            output = model_f(params, x)
            loss = get_cross_entropy(output, z)
            params = opt.step(loss, params)
    return flatten(params)


Intersection size: 576/576 (1.00)
Added 576 elements from intersection
Added 0 elements from group 0: S(4)
Taking random subset: 230/576 (0.40)
Train set size: 230/576 (0.40)


/tmp/ipykernel_96809/1958598494.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.make_functional` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.func.functional_call` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  model_f, init_params = functorch.make_functional(model)


In [3]:
train_jac = t.func.jacrev(partial(train, epochs=2))(flatten(init_params))
train_jac.shape

100%|██████████| 2/2 [00:00<00:00, 42.09it/s]


torch.Size([4376, 4376])

In [4]:
t.allclose(train_jac, t.eye(train_jac.shape[0]))

False

In [5]:
train_jac[:10, :10]

tensor([[ 9.9999e-01,  1.3763e-06, -4.0296e-06, -1.3249e-06,  1.9222e-06,
         -2.1517e-06, -3.1543e-07, -4.1833e-06,  1.4107e-06, -5.3752e-07],
        [-4.8221e-04,  9.9739e-01,  3.9119e-04,  3.7268e-05,  7.1354e-04,
          3.7365e-04, -1.7262e-04,  8.6500e-05, -8.3316e-06, -4.8661e-04],
        [-1.8565e-05,  2.0593e-06,  9.9971e-01, -1.7941e-05, -6.4639e-05,
         -1.9601e-05,  4.4963e-05, -7.7958e-06,  1.1871e-05, -4.6507e-05],
        [-7.2219e-06, -5.2852e-06, -4.3495e-06,  9.9997e-01,  1.3083e-05,
          7.3948e-06,  6.3212e-06,  8.7692e-07,  7.7940e-06, -4.9355e-06],
        [ 1.8482e-05,  2.4371e-05, -6.2865e-05,  3.7429e-05,  9.9972e-01,
         -1.0835e-05, -3.7380e-05,  3.7753e-05, -1.2660e-05,  5.4999e-05],
        [ 1.7986e-05,  1.9715e-05, -1.1433e-05,  1.5340e-05, -2.9916e-05,
          9.9988e-01,  2.0188e-05,  2.9279e-06, -7.8407e-06, -6.9179e-06],
        [ 2.2611e-06,  1.6279e-06,  5.5052e-06,  2.8162e-08, -6.1209e-07,
          4.5893e-08,  9.9999e-0

In [32]:
[t.shape for t in init_params]

[torch.Size([1, 24, 32]),
 torch.Size([1, 24, 32]),
 torch.Size([1, 32, 32]),
 torch.Size([1, 32, 32]),
 torch.Size([1, 32, 24]),
 torch.Size([1, 24])]

In [17]:
params = train(init_params, epochs=1000)
x, z = next(iter(train_loader))
x = x.to(device)
z = z.to(device)
output = model_f(params, x)
get_cross_entropy(output, z)

100%|██████████| 1000/1000 [00:07<00:00, 125.18it/s]


tensor([0.0008], grad_fn=<MeanBackward1>)

tensor([0.0008], grad_fn=<MeanBackward1>)